In [58]:
import numpy as np
import pandas as pd
import pathlib
import plotly.graph_objects as go
from neuralprophet import NeuralProphet

from sktime.forecasting.base import ForecastingHorizon

# Multivariate neural prophet
Use NeuralProphet to generate baseline quantile forecasts. 

In [59]:
EXPERIMENT_NAME = "neuralprophet_univariate"

In [60]:
df = pd.read_csv("../data/processed_data_inputs/Connex.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df.head()

prediction_length = 12
context_length=52
set_freq = 'W'

df = df.resample(set_freq).sum()

## As a place-holder using parallel CONNEX datasets as covariates 
# all_data = df.drop(columns=target_categories)
main_data = df
main_data.index.name = None


In [61]:
#Collect Government of Ontario COVID-19 datasets 
covid_data = pd.read_csv("../data/processed_data_inputs/COVID19.csv", index_col=0)
covid_data.index = pd.to_datetime(covid_data.index)
covid_data = covid_data.resample(set_freq).sum()

covid_data
#Collect google trends data 
google_data = pd.read_csv("../data/processed_data_inputs/GoogleTrends.csv", index_col=0)
google_data.index = pd.to_datetime(google_data.index)


#Collect Statistics Canada data
statcan_data = pd.read_csv("../data/processed_data_inputs/StatCan.csv", index_col=0)
statcan_data.index = pd.to_datetime(statcan_data.index)
statcan_data = statcan_data.resample(set_freq).interpolate(method='linear')


#Combine the three sources of covariates
all_covs = google_data.join(covid_data, how='left')

all_covs = all_covs.join(statcan_data, how='left')


# Check for duplicate indices
duplicate_indices = all_covs.index.duplicated(keep='first')
# print("Duplicate Indices:")
# print(all_covs[duplicate_indices])

# Drop duplicate indices
all_covs = all_covs[~duplicate_indices]
all_covs

,anxiety,suicide,fear,loneliness,allmentaldistress,mortgage,mortgageloan,unemployment,foodbank,welfare,...,Energy,Food,Gasoline,Goods,Health and personal care,"Household operations, furnishings and equipment","Recreation, education and reading",Services,Shelter,Transportation
2018-01-07,82.0,33.0,38.0,48.0,281.0,41.0,56.0,15.0,20.0,59.0,...,161.969231,144.692308,181.738462,118.923077,127.769231,125.230769,118.192308,146.923077,141.253846,137.976923
2018-01-14,84.0,32.0,38.0,100.0,354.0,48.0,70.0,14.0,21.0,62.0,...,162.488462,144.884615,182.776923,119.046154,127.838462,125.311538,118.134615,146.996154,141.307692,138.203846
2018-01-21,87.0,26.0,35.0,70.0,312.0,40.0,43.0,15.0,18.0,80.0,...,163.007692,145.076923,183.815385,119.169231,127.907692,125.392308,118.076923,147.069231,141.361538,138.430769
2018-01-28,92.0,27.0,35.0,43.0,294.0,37.0,44.0,14.0,23.0,93.0,...,163.526923,145.269231,184.853846,119.292308,127.976923,125.473077,118.019231,147.142308,141.415385,138.657692
2018-02-04,80.0,23.0,35.0,47.0,273.0,34.0,76.0,14.0,11.0,69.0,...,164.046154,145.461538,185.892308,119.415385,128.046154,125.553846,117.961538,147.215385,141.469231,138.884615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-29,81.0,18.0,42.0,47.0,263.0,48.0,74.0,13.0,50.0,65.0,...,203.800000,183.500000,216.600000,139.500000,143.600000,134.700000,122.700000,169.700000,172.800000,164.300000
2023-02-05,79.0,17.0,41.0,25.0,227.0,42.0,55.0,12.0,21.0,66.0,...,203.800000,183.500000,216.600000,139.500000,143.600000,134.700000,122.700000,169.700000,172.800000,164.300000
2023-02-12,78.0,16.0,39.0,56.0,254.0,38.0,70.0,10.0,48.0,47.0,...,203.800000,183.500000,216.600000,139.500000,143.600000,134.700000,122.700000,169.700000,172.800000,164.300000
2023-02-19,82.0,19.0,37.0,40.0,239.0,42.0,85.0,8.0,35.0,52.0,...,203.800000,183.500000,216.600000,139.500000,143.600000,134.700000,122.700000,169.700000,172.800000,164.300000


In [62]:
all_data = main_data.join(all_covs, how='left')
all_data = all_data.loc[all_data.index >= '2018-01-01']
all_data = all_data.fillna(0)

all_data.index = pd.to_datetime(all_data.index)
all_data

,IS_MENTAL_HEALTH,IS_SUBSTANCE_ABUSE,IS_PROBLEM_GAMBLING,IS_OTHER,IS_MENTAL_HEALTH_Concurrent Disorder Clients,IS_SUBSTANCE_ABUSE_Concurrent Disorder Clients,IS_PROBLEM_GAMBLING_Concurrent Disorder Clients,IS_OTHER_Concurrent Disorder Clients,IS_MENTAL_HEALTH_Concurrent Disorders,IS_SUBSTANCE_ABUSE_Concurrent Disorders,...,Energy,Food,Gasoline,Goods,Health and personal care,"Household operations, furnishings and equipment","Recreation, education and reading",Services,Shelter,Transportation
2018-01-07,1367,701,111,243,69.0,69.0,7.0,0.0,0.0,0.0,...,161.969231,144.692308,181.738462,118.923077,127.769231,125.230769,118.192308,146.923077,141.253846,137.976923
2018-01-14,1486,832,170,218,84.0,94.0,6.0,0.0,0.0,0.0,...,162.488462,144.884615,182.776923,119.046154,127.838462,125.311538,118.134615,146.996154,141.307692,138.203846
2018-01-21,1637,911,206,206,46.0,47.0,4.0,0.0,0.0,0.0,...,163.007692,145.076923,183.815385,119.169231,127.907692,125.392308,118.076923,147.069231,141.361538,138.430769
2018-01-28,1740,888,164,158,86.0,89.0,1.0,0.0,0.0,0.0,...,163.526923,145.269231,184.853846,119.292308,127.976923,125.473077,118.019231,147.142308,141.415385,138.657692
2018-02-04,1739,892,171,144,87.0,87.0,19.0,0.0,0.0,0.0,...,164.046154,145.461538,185.892308,119.415385,128.046154,125.553846,117.961538,147.215385,141.469231,138.884615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-07,2454,1208,318,12,0.0,0.0,0.0,0.0,283.0,283.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-05-14,1569,1076,286,13,0.0,0.0,0.0,0.0,208.0,208.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-05-21,3286,2207,709,21,0.0,0.0,0.0,0.0,589.0,589.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-05-28,2127,1259,237,17,0.0,0.0,0.0,0.0,283.0,283.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [63]:
#Define target categories 
target_categories = ['IS_MENTAL_HEALTH', 'IS_SUBSTANCE_ABUSE']
target_categories

['IS_MENTAL_HEALTH', 'IS_SUBSTANCE_ABUSE']

In [64]:
main_df = main_data[target_categories]
main_df.tail()

,IS_MENTAL_HEALTH,IS_SUBSTANCE_ABUSE
2023-05-07,2454,1208
2023-05-14,1569,1076
2023-05-21,3286,2207
2023-05-28,2127,1259
2023-06-04,882,466


## Forecasting function

AutoARIMA is a univariate forecasting method. This function expects that `df` has a time series index (monthly samples) and the target data column is the 0th index in the dataframe (see cell line 15).

In [65]:
def reformat_outputs(df):
    print(category)
    return df.rename(columns={'ds':'REF_DATE', 'y':category}).set_index('REF_DATE')

def format_neuralprophet_inputs(df):
    #Modify to act as an input to neural prophet 
#     cat_name = str.split(category, '_x')[0]
    cat_name = category

    print(cat_name)
    return df.reset_index().rename(columns={'index':'ds', cat_name:'y'})

def get_forecast_NP_weekly(
    df,
    covariates,
    context_length,
    prediction_length,
    cutoff_date,
):
    
    global context_df
    global np_future_df
        
    #Establish cutoff date
    if cutoff_date is None:
        date = df.index[-1]
    else:    
        date = pd.to_datetime(cutoff_date)
        print('cutoff date is {}'.format(date))
    
    
    #Combine food category dataframe and covariates
    df = covariates
#     df = df.join(df, covariates, how ='left')

    #Resample to be weekly
    df = df.resample(set_freq).mean().interpolate()
    quantiles = [0.01, 0.05, 0.1, 0.25, 0.75, 0.9, 0.95, 0.99]
    fh = ForecastingHorizon(list(range(1, prediction_length + 1)))
    
    context_df = df.loc[df.index <= date]

    if 'ds' not in df:
        context_df = format_neuralprophet_inputs(context_df) #Format inputs for neural prophet


    #Initialize the neural prophet model
    model = NeuralProphet(
        n_lags = context_length,
        n_forecasts=prediction_length,
        quantiles=quantiles
    )
    
    # Add each column (except 'y') as a regressor
#     for covariate in context_df.columns:
#         if covariate not in ['ds', 'y']:
#             print('adding regressor for {}'.format(covariate))
#             model.add_lagged_regressor(covariate, n_lags=n_lags)

    context_df = context_df[['y','ds']]

    #Fit NP model 
    
    print(context_df.columns)
    model.fit(context_df, freq=set_freq)

    #Make predictions using quantiles
    np_future_df = model.make_future_dataframe(context_df, periods=prediction_length)
    fc_series = model.predict(np_future_df, decompose=False, raw=False)

    fc_series

    #Format quantile-based outputs to be consistent with other experiments 
    all_quantiles= {}
    all_pred = pd.DataFrame
    pred_num = prediction_length-1
    for pred_num in range(1,prediction_length+1):
        all_quantiles= {}
        for q in quantiles:
            all_quantiles[f"q_{q}"] = (fc_series['yhat{} {}%'.format(pred_num,q*100)])
            all_quantiles[f"q_{0.5}"] = (fc_series['yhat{}'.format(pred_num)])

            check = pd.DataFrame(all_quantiles)
            check = check[check.notna().all(axis=1)]

        if pred_num == 1: #First run through
            all_pred = check
        else: #All subsequent runs
            all_pred = all_pred.append(check, ignore_index=True)

    forecast_min_date = date + pd.DateOffset(weeks=1)
    forecast_max_date = date + pd.DateOffset(weeks=1 * prediction_length + 1)
    forecast_date_index = pd.date_range(
        forecast_min_date, forecast_max_date, freq=set_freq
    )

    forecast_df = pd.DataFrame(all_pred).set_index(
        forecast_date_index
    ).resample(set_freq).mean()

    forecast_df = forecast_df[sorted(forecast_df.columns)]

    context_df = context_df.loc[context_df.ds >= (date - pd.DateOffset(weeks=context_length))]

    if 'ds' in context_df.columns: #Reformat outputs to match other models
        context_df = reformat_outputs(context_df)

    return context_df, forecast_df

## Load data

Since this is a simple univariate / autoregressive model, we're only loading the 'target' CPI variables here.

## Define experiment cutoff dates

Our experiment design uses 6 annual cutoff dates that simulate the generation of forecast once per year over the last 6 years. We'll comsume data up to each cutoff date to fit/train models, and then evaluate over the next 18 months. In this notebook, we're only concerned with producing the retrospective forecasts and we'll do the analysis all together in another notebook.

In [66]:
report_sim_dates = open("../data/utils/experiment_cutoff_dates.txt", 'r').read().split()
report_sim_dates

['2020-3-1',
 '2020-9-27',
 '2021-3-14',
 '2021-9-5',
 '2021-12-12',
 '2022-4-3',
 '2022-7-31']

## Plot forecast range with context

We could add different elements to plots including some error analysis, emphasis on different quantiles, etc.

In [67]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def plot_quantile_forecast(category, context_df, forecast_df, actual_df, cutoff_date, save_path=None, show_plots=True):

    fig, ax = plt.subplots(figsize=(10,6))

    # Context
    context_df = pd.concat((context_df, actual_df))
    ax.plot(context_df.index, context_df.values, color='black', label='Historical CPI')

    # Confidence range between 0.01 and 0.99 quantiles
    ax.fill_between(
        forecast_df.index,
        forecast_df[f"q_0.05"],
        forecast_df[f"q_0.95"],
        facecolor='purple',
        alpha=0.5,
        label='95% Confidence'
    )

    # Add a line trace for the median
    ax.plot(forecast_df.index, forecast_df[f"q_0.5"], color='purple', label='Median Forecast')

    # Update the layout as needed
    ax.set_title(f'{category}\nRetrospective Forecast - {cutoff_date}')
    ax.set_xlabel('Date')
    ax.set_ylabel('CPI (% 2002 Prices)')
    ax.axvline(pd.to_datetime(cutoff_date), label="Cutoff Date", color='black', ls='--', ms=1, alpha=0.5)
    ax.legend()
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.xticks(rotation=45)

    # Show the figure
    plt.grid(axis='y')

    if show_plots:
        plt.show()

    # Save the figure if the path is specified
    if save_path:
        fig.savefig(save_path, dpi=300 if save_path.endswith("png") else None) # High res for png


## Main experiment loop

In [73]:
for cutoff_date in report_sim_dates:
    forecast_output_dir = f"./output/experiments/{EXPERIMENT_NAME}/{cutoff_date}/forecasts"
    plot_output_dir = f"./output/experiments/{EXPERIMENT_NAME}/{cutoff_date}/plots"
    pathlib.Path(forecast_output_dir).mkdir(parents=True, exist_ok=True)
    pathlib.Path(plot_output_dir).mkdir(parents=True, exist_ok=True)
    for category in target_categories:
        n_lags = context_length
        covariates =  all_data[all_data.columns.values]
        covariates[category] = all_data[[category]]

        context_df, forecast_df = get_forecast_NP_weekly(
        main_df[[category]],  # Indexing this way to get a dataframe with 1 column as opposed to a series
        covariates,                #Context df with covariates
        context_length=context_length,      # Context length 
        prediction_length=prediction_length,
        cutoff_date=cutoff_date    # Training / fitting cutoff date
        )
        forecast_df.to_csv(f"{forecast_output_dir}/{category}.csv")
        plot_quantile_forecast(
            category=category,         # The target category name
            context_df=context_df,     # Historical data to plot
            forecast_df=forecast_df,   # Quantile forecast dataframe
            actual_df=main_df[category].loc[[date for date in main_df.index if date in main_df.index]], # 'actual' data to plot against forecast
            save_path=f"{plot_output_dir}/{category}.svg", 
            cutoff_date=cutoff_date,
            show_plots=False
        )

WARNING - (py.warnings._showwarnmsg) - /h/kupfersk/.local/lib/python3.8/site-packages/sktime/forecasting/base/_fh.py:100: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  values = pd.Int64Index(values, dtype=int)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.115% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True

cutoff date is 2020-03-01 00:00:00
IS_MENTAL_HEALTH
Index(['y', 'ds'], dtype='object')


Finding best initial lr:   0%|          | 0/208 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /h/kupfersk/.conda/envs/new_env/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")



AttributeError: 'LearningRateFinder' object has no attribute 'optimal_lr'